<a href="https://colab.research.google.com/github/Hoang-TamSE/Logistic-Regression/blob/main/StochasticLogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/anminhhung/small_dog_cat_dataset

In [ ]:

# Import libraries
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow



In [ ]:
cat_train = glob.glob("small_dog_cat_dataset/train/cats/*.jpg")
dog_train = glob.glob("small_dog_cat_dataset/train/dogs/*.jpg")
cat_test = glob.glob("small_dog_cat_dataset/test/cats/*.jpg")
dog_test = glob.glob("small_dog_cat_dataset/test/dogs/*.jpg")

In [ ]:
def image_to_list(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Resize image
    image = cv2.resize(image, (200, 200))

    # Convert the image to grayscale if needed
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Flatten the image array
    pixel_values = image.flatten()


    return np.array(pixel_values)

In [ ]:
# Show 1 image for instance
im_path_ex = "small_dog_cat_dataset/train/cats/cat.10000.jpg"
im_ex = cv2.imread(im_path_ex)
cv2_imshow(im_ex)
plt.show()

In [ ]:
# Check the form of the data
print(image_to_list(im_path_ex))
print(len(image_to_list(im_path_ex)))
print(type(image_to_list(im_path_ex)))

In [ ]:
# The result can't change when running again
np.random.seed(2)

# Length of cat_train
print(len(cat_train))

In [ ]:
# Training with cats and dogs
ones_array = np.ones(2000)
ones_array = ones_array.reshape(1, -1)
cat_train_x = np.array([image_to_list(image_path) for image_path in cat_train])
dog_train_x = np.array([image_to_list(image_path) for image_path in dog_train])
train_x = np.concatenate((cat_train_x, dog_train_x), axis = 0)
train_x = np.column_stack(train_x)
train_x = np.vstack((ones_array, train_x))
print(train_x.shape)
# Cat is labeled 1
cat_train_y = np.ones(1000)
# Dog is labeled 0
dog_train_y = np.zeros(1000)
# Concatenate cat and dog
train_y = np.concatenate((cat_train_y, dog_train_y), axis=0)
print(train_y.shape)


In [ ]:
# Logistic regression
def sigmoid_function(s):
    if s < 0:
      return np.exp(s) / (1 + np.exp(s))
    else:
      return 1 / (1 + np.exp(-s))

def logistic_sigmoid_regression(x, y, weight_init, learning_rate = 0.05, error = 1e-4, num_iteration = 20000):
    weight = [weight_init]    
    it = 0
    num_epoch = x.shape[1]
    num_para = x.shape[0]
    count = 0
    check_weight_after = 20
    while count < num_iteration:
        # mix data 
        mix_id = np.random.permutation(num_epoch)
        for i in mix_id:
            xi = x[:, i].reshape(num_para, 1)
            yi = y[i]
            zi = sigmoid_function(np.dot(weight[-1].T, xi))
            weight_new = weight[-1] + learning_rate*(yi - zi)*xi
            count += 1
            # stopping criteria
            if count % check_weight_after == 0:                
                if np.linalg.norm(weight_new - weight[-check_weight_after]) < error:
                    return weight
            weight.append(weight_new)
    return weight

np.random.seed(2023)
weight_init = np.random.randn(120001, 1)
weight = logistic_sigmoid_regression(train_x, train_y, weight_init)
print(weight[-1])
print(weight[-1].T.shape)
print(cat_train_x.shape)

In [ ]:
evaluate_metric = 0.0
for image_path in cat_test:
    one_cat_test_x = image_to_list(image_path)
    one_cat_test_x = np.concatenate(([1], one_cat_test_x))
    one_cat_test_x = one_cat_test_x[:, np.newaxis]
    evaluate_metric = evaluate_metric + sigmoid_function(np.dot(weight[-1].T, one_cat_test_x))

for image_path in dog_test:
    one_dog_test_x = image_to_list(image_path)
    one_dog_test_x = np.concatenate(([1], one_dog_test_x))
    one_dog_test_x = one_dog_test_x[:, np.newaxis]
    evaluate_metric = evaluate_metric + 1-sigmoid_function(np.dot(weight[-1].T, one_dog_test_x))
evaluate_metric = evaluate_metric / 2000.0
print(evaluate_metric)
